<font color='red'>For this to succeed you must have run the examples/hmc.ipynb!</font>

We reuse the previously-generated example ensemble.

# Setup

In [ ]:
import numpy as np
import torch

print(f'{torch.version.git_version=}')
print(f'{torch.version.hip=}')
print(f'{torch.version.debug=}')
print(f'{torch.version.cuda=}')

cpu = torch.device('cpu')

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'Using GPU {torch.cuda.current_device()}')
    torch.set_default_tensor_type('torch.cuda.DoubleTensor')
else:
    device = torch.device('cpu')
    torch.set_default_tensor_type(torch.DoubleTensor)

In [ ]:
import numpy as np
import h5py as h5

import tdg
import tdg.HMC as HMC
import tdg.plot as visualize

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
def plot_history(ensemble, history=None, label=None):
    
    if history is None:
        history = visualize.History(7)
    
    history.plot(ensemble.S.real,              0, x=ensemble.index, label=label)
    history.plot(ensemble.N.real,              1, x=ensemble.index, label=('fermionic' if not label else label))
    history.plot(ensemble.Kinetic.real,        2, x=ensemble.index, label=label)
    history.plot(ensemble.Potential.real,      3, x=ensemble.index, label=label)
    history.plot(ensemble.InternalEnergy.real, 4, x=ensemble.index, label=label)
    history.plot(ensemble.Contact.real,        5, x=ensemble.index, label=label)
    
    history.plot(ensemble.Spin[:,0].real,      6, x=ensemble.index, label=label)
    history.plot(ensemble.Spin[:,1].real,      6, x=ensemble.index, label=label)
    history.plot(ensemble.Spin[:,2].real,      6, x=ensemble.index, label=label)

    history.ax[0,0].set_ylabel('S')
    history.ax[1,0].set_ylabel('N')
    history.ax[2,0].set_ylabel('K')
    history.ax[3,0].set_ylabel('V')
    history.ax[4,0].set_ylabel('U')
    history.ax[5,0].set_ylabel('Contact')
    history.ax[6,0].set_ylabel('Spin')
    
    return history

# Read the ensemble

In [ ]:
storage = 'hmc.h5'

In [ ]:
with h5.File(storage, 'r') as f:
    ensemble = tdg.ensemble.GrandCanonical.from_h5(f['/example'])

In [ ]:
plot_history(ensemble)

# Binning

We've got 2000 configurations.  We can drop some for thermalization, keep only every 4th and bin every (remaining) 4th to help keep autocorrelation down.

In [ ]:
thermalization=200
frequency=6
binning=4

In [ ]:
binned = ensemble.cut(thermalization).every(frequency).binned(binning)

How many samples should we have, given that the ensemble started with 2000 configurations?

In [ ]:
expected = (
    (
        len(ensemble)-thermalization # cut
    )/frequency # every
) / binning # binning
print(f'The binning has {len(binned)} samples, while we expected {expected}')

Let's compare the binned samples with the original ensemble.

In [ ]:
viz = visualize.ScatterTriangle(2)
viz.plot(
    (ensemble.N_bosonic.real,
     ensemble.N.real
    ))
viz.plot(
    (binned.N_bosonic.real,
     binned.N.real
    ))

We can see how the binning averages samples across Markov chain time.

In [ ]:
viz = plot_history(ensemble, label='ensemble')
viz = plot_history(binned, history=viz, label='binned')

viz.ax[0,0].legend()

# Bootstrapping

Let us compare different binnings and rest assured that our uncertainty estimate is big enough once the bootstrap errors stabilize.

In [ ]:
def naive_estimate(obs):
    return f'{obs.real.mean():.2f}±{obs.real.std()/torch.sqrt(torch.tensor(obs.shape[0])):.2f}'

def bootstrap_estimate(obs):
    return f'{obs.real.mean():.2f}±{obs.real.std():.2f}'

We repeat the binning for different bin widths (not dropping any measurements).

We see that the uncertainties have stabilized: the binnings of widths 16 and 24 give approximately the same uncertainty.

In [ ]:
print(f"ENSEMBLE")
# This obviously uses an extremely naive estimate for the uncertainty!
print(f"fermionic N: {naive_estimate(ensemble.N)}")
print(f"bosonic   N: {naive_estimate(ensemble.N_bosonic)}")
print(f"action:      {naive_estimate(ensemble.S)}")
print(f"Contact:     {naive_estimate(ensemble.Contact)}\n")

viz = visualize.ScatterTriangle(4, figsize=(12,12), labels=('fermionic N', 'bosonic N', 'action', 'Contact',))
for width in [1, 2, 4, 8, 16, 24]:
    binned = ensemble.cut(thermalization).binned(width)
    bootstrap = binned.bootstrapped()
    viz.plot(
        (bootstrap.N_bosonic.real,
         bootstrap.N.real,
         bootstrap.S.real,
         bootstrap.Contact.real,
        ))
    
    print(f"BOOTSTRAP {width=} ({len(binned)} bins)")
    print(f"fermionic N: {bootstrap_estimate(bootstrap.N)}")
    print(f"bosonic   N: {bootstrap_estimate(bootstrap.N_bosonic)}")
    print(f"action:      {bootstrap_estimate(bootstrap.S)}")
    print(f"Contact:     {bootstrap_estimate(bootstrap.Contact)}\n")
